In [40]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/project4/spoiler_df')

In [41]:
df.head(2)

,movie_id,summary,score,summary_len,synopsis_len,release_year,user_id,is_spoiler,review,score_by_user,review_year
0,tt0105112,former cia analyst jack ryan is in england wit...,6.9,1034,8605,1992,ur23055365,False,patriot games is a great movie with a very wel...,8,2014
1,tt0105112,former cia analyst jack ryan is in england wit...,6.9,1034,8605,1992,ur0020866,False,phillip noyce takes the helm for the first tim...,9,2009


In [42]:
# movie_id
df['movie_id'].value_counts(normalize=True)

tt0468569    0.009197
tt0111161    0.008161
tt0167260    0.005200
tt0137523    0.004726
tt0110912    0.003805
               ...   
tt0101301    0.000029
tt6868216    0.000025
tt0107719    0.000023
tt6294822    0.000021
tt0201265    0.000008
Name: movie_id, Length: 1418, dtype: float64

In [43]:
# user_id
df['user_id'].value_counts(normalize=True)

ur2898520     0.002248
ur4248714     0.001738
ur60028700    0.001362
ur0453068     0.001326
ur20552756    0.001219
                ...   
ur5137058     0.000002
ur19454873    0.000002
ur23294511    0.000002
ur1654432     0.000002
ur0238573     0.000002
Name: user_id, Length: 247550, dtype: float64

In [44]:
df['summary_len'] = df['summary'].apply(lambda x: len(x))
df['review_len'] = df['review'].apply(lambda x: len(x))

In [45]:
df = df[df['summary_len'] < 1000]
df = df[df['review_len'] < 2000]

In [46]:
print(df.shape)

(369127, 12)


In [47]:
df.head(3)

,movie_id,summary,score,summary_len,synopsis_len,release_year,user_id,is_spoiler,review,score_by_user,review_year,review_len
100,tt1204975,billy michael douglas paddy robert de niro arc...,6.6,633,5371,2013,ur47921155,False,this movie shows how we can become as we age ...,10,2013,638
102,tt1204975,billy michael douglas paddy robert de niro arc...,6.6,633,5371,2013,ur20781285,False,I saw the movie last vegas last night with rut...,8,2013,791
103,tt1204975,billy michael douglas paddy robert de niro arc...,6.6,633,5371,2013,ur10327852,True,when you have the calibre of actors such as mi...,5,2014,1019


#### 토큰화

In [48]:
pip install spacy

In [49]:
pip install nlp

     |████████████████████████████████| 1.7MB 12.4MB/s 
     |████████████████████████████████| 245kB 44.1MB/s 


In [50]:
pip install -U spacy

     |████████████████████████████████| 12.8MB 11.9MB/s 
     |████████████████████████████████| 9.1MB 38.3MB/s 
     |████████████████████████████████| 51kB 4.8MB/s 
     |████████████████████████████████| 460kB 43.6MB/s 
     |████████████████████████████████| 1.1MB 43.7MB/s 
     |████████████████████████████████| 122kB 28.4MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-cp37-none-any.whl size=107098 sha256=f6497627083cb1b90de66aacb1941f8c72a15a3475ac1f9102c1c5a7b587d635
  Stored in directory: /root/.cache/pip/wheels/18/88/7c/f06dabd5e9cabe02d2269167bcacbbf9b47d0c0ff7d6ebcb78
Successfully built smart-open
  Found existing installation: smart-open 5.0.0
    Uninstalling smart-open-5.0.0:
      Successfully uninstalled smart-open-5.0.0
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly

In [51]:
pip install https://github.com/pyinstaller/pyinstaller/archive/develop.zip

     - 7.1MB 6.1MB/s
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 184kB 9.9MB/s 
  Created wheel for pyinstaller: filename=pyinstaller-5.0.dev0-cp37-none-any.whl size=415050 sha256=dbab9cddf53e2669ec061169c0adfcc26900290dea5642a704dfbdf99176e8ff
  Stored in directory: /tmp/pip-ephem-wheel-cache-lupi1uyt/wheels/1e/5b/0a/b7b241e8886f8f6d4c41177dd91cdb275f0cda6cd9dde93269
Successfully built pyinstaller


In [52]:
!pip install -U pip setuptools wheel
!pip install -U spacy

     |████████████████████████████████| 1.6MB 12.4MB/s 
  Using cached https://files.pythonhosted.org/packages/11/1b/7012b145cb228aed20f9b2b8b259df49e7963d900799ea44791f54d06ab9/setuptools-56.1.0-py3-none-any.whl
Requirement already up-to-date: wheel in /usr/local/lib/python3.7/dist-packages (0.36.2)
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
  Found existing installation: setuptools 56.0.0
    Uninstalling setuptools-56.0.0:
      Successfully uninstalled setuptools-56.0.0


In [53]:
import spacy
from spacy.tokenizer import Tokenizer

In [54]:
!python -m spacy download en_core_web_lg

2021-05-05 10:42:09.971039: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 778.8 MB 18 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [55]:
nlp = spacy.load('en_core_web_lg')

In [56]:
tokenizer = Tokenizer(nlp.vocab)

In [57]:
df['sum_tokens'] = df['summary'].apply(lambda x: [ token.text for token in tokenizer(x)])

In [58]:
df['re_tokens'] = df['review'].apply(lambda x: [ token.text for token in tokenizer(x)])

In [59]:
from gensim.models import KeyedVectors

In [60]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [61]:
EMBEDDING = '/content/drive/MyDrive/project4/GoogleNews-vectors-negative300.bin.gz'

In [62]:
# embedding
vocab = dict()
inverse_vocab = ['<nan>']
stops = set(stopwords.words('english'))
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING, binary=True)

In [63]:
for index, row in df.iterrows():
    for col in ['sum_tokens', 're_tokens']:
        text_to_list = []
        
        for word in row[col]:
            
            # 이번 단어가 불용어이면 패스
            if (word in stops) and (word not in word2vec.vocab):
                continue
            
            # 단어가 vocab에 없을 경우 추가
            if word not in vocab:
                vocab[word] = len(inverse_vocab)
                text_to_list.append(len(inverse_vocab))
                inverse_vocab.append(word)
            else:
                text_to_list.append(vocab[word])
         
        # 토큰 리스트를 숫자 리스트로 교체
        df.at[index, col] = text_to_list

In [64]:
np.random.seed(21)
embedding_dim = 300
embeddings = 1 * np.random.randn(len(vocab) + 1, embedding_dim)
embeddings[0] = 0

for word, index in vocab.items():
    if word in word2vec.vocab:
        embeddings[index] = word2vec.word_vec(word)

In [65]:
import itertools
from keras.preprocessing.sequence import pad_sequences

#### 저장

In [68]:
from sklearn.model_selection import train_test_split

In [69]:
# train, validation, test dataset
target = 'is_spoiler'

train, test = train_test_split(df,
                               test_size=0.2,
                               random_state=42,
                               stratify=df[target])

train, val = train_test_split(train,
                              test_size=0.25,
                              random_state=42,
                              stratify=train[target])


print(train.shape, val.shape, test.shape)

(221475, 14) (73826, 14) (73826, 14)


In [70]:
import pickle
import gzip

In [83]:
# 저장 함수 
def save_as_pickle(data, file_name):
    base_path = '/content/drive/MyDrive/project4/IMDB'
    file_path = os.path.join(base_path, file_name)
    
    with gzip.open(file_path, 'wb') as f:
        pickle.dump(data, f)

In [84]:
df.head(1)

,movie_id,summary,score,summary_len,synopsis_len,release_year,user_id,is_spoiler,review,score_by_user,review_year,review_len,sum_tokens,re_tokens
100,tt1204975,billy michael douglas paddy robert de niro arc...,6.6,633,5371,2013,ur47921155,False,this movie shows how we can become as we age ...,10,2013,638,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[84, 85, 86, 87, 88, 89, 90, 91, 92, 88, 41, 7..."


In [85]:
import os

In [88]:
# feature, target dataset
col = ['score', 'release_year', 'summary_len', 'score_by_user', 'review_year','review_len']


save_as_pickle(train[col], 'train.pickle')
save_as_pickle(val[col], 'val.pickle')
save_as_pickle(test[col], 'test.pickle')
save_as_pickle(embeddings, 'embeddings.pickle')
save_as_pickle(vocab, 'vocab.pickle')

In [89]:
import itertools
from keras.preprocessing.sequence import pad_sequences

In [90]:
# 토큰 dataset zero padding 후 따로 저장 (feature / target)
target = 'is_spoiler'

X_train = {'summary': train['sum_tokens'],
           'review':train['re_tokens']}

X_val = {'summary': val['sum_tokens'],
         'review':val['re_tokens']}

X_test = {'summary': test['sum_tokens'],
          'review':test['re_tokens']}


In [91]:
y_train = train[target].values
y_val = val[target].values
y_test = test[target].values

In [92]:
# zero padding
max_len = max(
    train['sum_tokens'].map(lambda x: len(x)).max(),
    train['re_tokens'].map(lambda x: len(x)).max(),
    val['sum_tokens'].map(lambda x: len(x)).max(),
    val['re_tokens'].map(lambda x: len(x)).max(),
    test['sum_tokens'].map(lambda x: len(x)).max(),
    test['re_tokens'].map(lambda x: len(x)).max())

print(max_len)

for dataset, side in itertools.product([X_train, X_val, X_test], ['summary', 'review']):
    dataset[side] = pad_sequences(dataset[side], maxlen=max_len)

514


In [93]:
save_as_pickle(X_train, 'X_train_word.pickle')
save_as_pickle(X_val, 'X_val_word.pickle')
save_as_pickle(X_test, 'X_test_word.pickle')
save_as_pickle(y_train, 'y_train.pickle')
save_as_pickle(y_val, 'y_val.pickle')
save_as_pickle(y_test, 'y_test.pickle')
save_as_pickle(max_len, 'max_len.pickle')

In [94]:
# 문장들 
col = ['summary', 'review', 'is_spoiler']

save_as_pickle(train[col], 'train_sentences.pickle')
save_as_pickle(val[col], 'val_sentences.pickle')
save_as_pickle(test[col], 'test_sentences.pickle')